# MATLAB on Amazon ECR

1. [Introduction](#1.-Introduction)
   
2. [MATLAB on Amazon SageMaker](#2.-MATLAB-on-Amazon-SageMaker)

3. [Prerequisites](#3.-Prerequisites) 
    1. [Roles, Permissions and Docker Service](#3.1-Roles,-Permissions-and-Docker-Service) 
    2. [License Manager for MATLAB](#3.2-License-Manager-for-MATLAB) 
    3. [Dockerfile & dependencies](#3.3-Dockerfile-&-dependencies)
<br> <br>

4. [MATLAB docker image on ECR](#4.-MATLAB-docker-image-on-ECR)
    1. [Create docker image from Dockerfile](#4.1-Create-docker-image-from-Dockerfile) 
    2. [Push MATLAB image to ECR](#4.2-Push-MATLAB-image-to-ECR)

## 1. Introduction 

[Amazon SageMaker](https://aws.amazon.com/sagemaker/) helps data scientists and developers to prepare, build, train, and deploy high-quality machine learning (ML) models quickly by bringing together a broad set of capabilities purpose-built for ML.

## 2. MATLAB on Amazon SageMaker

This notebook will guide you through an example that builds Docker container for SageMaker with MATLAB, which can be used for processing, training and inference.

This notebook shows how you can:

1. Build a Docker container with MATLAB by using the [official docker hub MATLAB image](https://hub.docker.com/r/mathworks/matlab-deep-learning).  
2. Publish the built docker container to [Amazon ECR](https://aws.amazon.com/ecr/), from where it can be used by SageMaker jobs.

## 3. Prerequisites

### 3.1 Roles, Permissions and Docker Service

To get started, we'll import the Python libraries we need, and set up the environment with a few prerequisites for permissions and configurations.

In [ ]:
from sagemaker import get_execution_role
import pandas as pd
import sagemaker
import boto3
import os

role = get_execution_role()
print(role)

Beacuse we would be pulling the [matlab-deep-learning docker image](https://hub.docker.com/r/mathworks/matlab-deep-learning) (which has a compressed size of 7.89 GB), we need to change the default docker location to the EBS volume that we mounted. 

We stop the docker service, move the default docker directory from `/var/lib/docker` to `/home/ec2-user/Sagemaker/docker`, and then start the docker service again.

In [ ]:
# Delete if the mount already exists
!sudo rm -rf /home/ec2-user/SageMaker/docker/

In [ ]:
!mkdir -p /home/ec2-user/SageMaker/docker/
!sudo service docker stop
!sudo mv /var/lib/docker/ /home/ec2-user/SageMaker/docker/
!sudo ln -s /home/ec2-user/SageMaker/docker/ /var/lib/docker
!sudo service docker start

### 3.2 License Manager for MATLAB

Follow the steps in the GitHub repo to launch the License Manager for MATLAB on AWS - https://github.com/mathworks-ref-arch/license-manager-for-matlab-on-aws. 

Once you have the License Manager up and running - note down the private IP Address of the License Manager. Our docker instance that  will run MATLAB, would communicate to the License Manager for licensing via the `MLM_LICENSE_FILE` flag.

### 3.3 Dockerfile & dependencies

In [ ]:
!mkdir -p matlab-docker

Create a Dockerfile which pulls MATLAB image from https://hub.docker.com/r/mathworks/matlab-deep-learning and adds a new entrypoint. The ENTRYPOINT command specifies the instruction that is to be executed when a Docker container starts.

In our Dockerfile, container runs the `main.m` script when it starts. You create this `main.m` script in next notebooks, when you run this Docker container.

In [ ]:
%%writefile matlab-docker/Dockerfile
FROM mathworks/matlab-deep-learning:r2021b
USER root
ENTRYPOINT ["matlab", "-batch", "train; exit"]

In [ ]:
!echo ==== Generated Dockerfile ====
!cat matlab-docker/Dockerfile

## 4. MATLAB docker image on ECR  

A Docker image with MATLAB needs to be available for SageMaker to use. 

In the following steps, we will:
* Build a MATLAB Deep Learning Container from Dockerfile in [Section 3.3]().
* Create an ECR respository, and push the container to it.

In [ ]:
# Setup variables for ECR repository
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name

# You can change sagemaker-demo-ecr to a different name, but if you do, make sure to change it in later steps too.
ecr_repository = 'sagemaker-demo-ecr'
tag = ':matlab'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

print("ECR Repository Name: ", ecr_repository+tag)
print("ECR Repository URI:", processing_repository_uri)

### 4.1 Create docker image from Dockerfile

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository$tag matlab-docker/

In [ ]:
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker image ls

### 4.2 Push MATLAB image to ECR

In [ ]:
# Creates the ECR Repository if it doesn't exist
!aws ecr describe-repositories --repository-names ${ecr_repository} || aws ecr create-repository --repository-name ${ecr_repository}

# Authorize Docker to publish to ECR
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

In [ ]:
# push MATLAB image to ECR
!docker push $processing_repository_uri

Note down the `processing_repository_uri`, as we would refer to the same ECR repository for running MATLAB in SageMaker in next example notebooks. 